In [ ]:
import numpy as np

from pymoo.problems import get_problem
from pymoo.visualization.fitness_landscape import FitnessLandscape
import pandas as pd
import seaborn as sns

In [ ]:
from pymoo.core.problem import Problem
def get_bbob(name, n_var=10, **kwargs):
    try:
        import cocoex as ex
    except:
        raise Exception("COCO test suite not found. \nInstallation Guide: https://github.com/numbbo/coco")

    args = name.split("-")
    suite = args[0]
    n_instance = int(args[2])
    n_function = int(args[1].replace("f", ""))

    assert 1 <= n_function <= 24, f"BBOB has 24 different functions to be chosen. {n_function} is out of range."

    suite_filter_options = f"function_indices: {n_function} " \
                           f"instance_indices: {n_instance} " \
                           f"dimensions: {n_var}"

    problems = ex.Suite(suite, "instances: 1-999", suite_filter_options)
    assert len(problems) == 1, "COCO problem not found."

    coco = problems.next_problem()

    return n_function, n_instance, coco
class BBOBProblem(Problem):

    def __init__(self, name, n_var, pf_from_file=True, **kwargs):
        self.function, self.instance, self.object = get_bbob(name, n_var)
        self.name = name
        self.pf_from_file = pf_from_file

        coco = self.object
        n_var, n_obj, n_ieq_constr = coco.number_of_variables, coco.number_of_objectives, coco.number_of_constraints
        xl, xu = coco.lower_bounds, coco.upper_bounds

        super().__init__(n_var=n_var,
                         n_obj=n_obj,
                         n_ieq_constr=n_ieq_constr,
                         xl=xl,
                         xu=xu,
                         **kwargs)

    def _calc_pareto_set(self, *args, **kwargs):
        if self.n_obj == 1:
            fname = '._bbob_problem_best_parameter.txt'

            self.object._best_parameter(what="print")
            ps = np.loadtxt(fname)
            os.remove(fname)

            return ps

    def _calc_pareto_front(self, *args, **kwargs):
        if self.pf_from_file:
            return Remote.get_instance().load("pymoo", "pf", "bbob.pf", to="json")[str(self.function)][str(self.instance)]
        else:
            ps = self.pareto_set()
            if ps is not None:
                return self.evaluate(ps)

    def _evaluate(self, X, out, *args, **kwargs):
        out["F"] = np.array([self.object(x) for x in X])
        return out
    
    def evaluate_basic(self, x, *args, **kwargs):
        return self.object(x)

    def __getstate__(self):
        d = self.__dict__.copy()
        d["object"] = None
        return d

    def __setstate__(self, d):
        self.__dict__.update(d)
        self.object = get_bbob(self.name, self.n_var)
        

In [ ]:
problem_id=2
instance_id=1
dim=2
p=BBOBProblem(f'bbob-{problem_id}-{instance_id}',dim)

In [ ]:
from ioh import get_problem, ProblemClass
f = get_problem(problem_id, instance_id, dim, ProblemClass.BBOB)
d=pd.read_csv(f'../data/processed/dim_{dim}/F{problem_id}_I{instance_id}_{dim}D.csv',compression='zip',index_col=0)
d['raw_y']=d['raw_y']+f.optimum.y

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap


problem=p
#scatter_df=d.query('algorithm in ["AugmentedAEO","OriginalAEO","SHADE"] and run==0')
#scatter_df=d.query('algorithm in ["OriginalSBO",	"OriginalBFO",	"OriginalFPA", "OriginalNMRA" ] and run==0')
scatter_df=d.query('algorithm in ["BaseDE",	"SADE",	"BaseGA" ] and run==0')
#scatter_df=d.query('algorithm in ["AugmentedAEO", "BaseDE"]')
color_column='algorithm'
n_samples=100
A = np.linspace(problem.xl[0], problem.xu[0], n_samples)
B = np.linspace(problem.xl[1], problem.xu[1], n_samples)
X = all_combinations(A, B)

F = np.reshape(problem.evaluate(X, return_values_of=["F"]), (n_samples, n_samples))

_X = X[:, 0].reshape((n_samples, n_samples))
_Y = X[:, 1].reshape((n_samples, n_samples))
_Z = F.reshape((n_samples, n_samples))

fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
surf = ax.plot_surface(_X, _Y, _Z, cmap="summer", alpha=1)
plt.tight_layout()
plt.savefig(f'figures_clustering_new/surface_bbob-{problem_id}-{instance_id}.pdf',bbox_inches='tight',pad_inches=0)
fig.colorbar(surf)
plt.show()
colors = ['black','blue',"red",'yellow']



# get colormap from seaborn
cmap = ListedColormap(sns.color_palette("mako", 256).as_hex())
for id, c_v in enumerate(scatter_df[color_column].unique()):
    for run in scatter_df['run'].unique():
        fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
        to_plot=scatter_df[scatter_df[color_column]==c_v]
        to_plot=to_plot.query('run==@run')
        sc=ax.scatter(to_plot['x0'], to_plot['x1'], to_plot['iteration'],c=to_plot['raw_y'], zorder=500, alpha=0.7, cmap=cmap)
        ax.set_xlabel('x0')
        ax.set_ylabel('x1')
        ax.set_zlabel('iteration')
        plt.legend(*sc.legend_elements(), bbox_to_anchor=(1.05, 1), loc=2)
        plt.tight_layout()
        plt.savefig(f'figures_clustering_new/scatterplot_bbob-{problem_id}-{instance_id}_{c_v}_run_{run}.pdf',bbox_inches='tight',pad_inches=0)
        print(c_v, run)
        plt.show()

fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
for id, c_v in enumerate(scatter_df[color_column].unique()):
    to_plot=scatter_df[scatter_df[color_column]==c_v]
    
    ax.scatter(to_plot['x0'], to_plot['x1'], to_plot['iteration'], color=colors[id], marker='x',zorder=500, alpha=0.5)
plt.savefig(f'figures_clustering_new/scatterplot_bbob-{problem_id}-{instance_id}.pdf',bbox_inches='tight',pad_inches=0)
plt.show()

In [ ]:
to_plot=d.query('algorithm=="HI_WOA" and run==0')
to_plot['color']=to_plot['algorithm'].apply(lambda x: 'b.' if x=="HI_WOA" else 'red')
x=MyFitnessLandscape(p, angle=(0, 0), _type="surface", scatter_x=to_plot['x0'],scatter_y=to_plot['x1'],scatter_z=to_plot['raw_y'], scatter_color=list(to_plot['color'].values) )
#sns.scatterplot(d.query('run==0 and iteration==0 and algorithm in ["LevyTWO","OriginalHC"]'), x='x0', y='x1', hue="algorithm",  palette='mako')
x.show()

In [ ]:
to_plot=d.query('algorithm=="OriginalWOA" and run==0')
x=MyFitnessLandscape(p, angle=(0, 0), _type="surface", scatter_x=to_plot['x0'],scatter_y=to_plot['x1'],scatter_z=to_plot['raw_y'] )
#sns.scatterplot(d.query('run==0 and iteration==0 and algorithm in ["LevyTWO","OriginalHC"]'), x='x0', y='x1', hue="algorithm",  palette='mako')
x.show()

In [ ]:
all_results=pd.concat(all_results)

In [ ]:
all_results.groupby(['algorithm1','algorithm2'])['p_value'].median().sort_values(ascending=False).tail(50)

In [ ]:
d=pd.read_csv('../data/processed/dim_2/F14_I1_2D.csv',compression='zip',index_col=0)

In [ ]:
for i in range(0,19):
    plt.figure()
    sns.scatterplot(d.query('run==0 and iteration==@i and algorithm in ["LevyTWO","OriginalHC"]'), x='x0', y='x1', hue="algorithm",  palette='mako')
    plt.show()

In [ ]:
for x in [f'x{i}' for i in range(0,dim)] + ['raw_y']:
    plt.figure()
    tt=d.query('algorithm in ["SHADE","L_SHADE"] and run==0').groupby(['algorithm','iteration'])[x].min().to_frame()
    sns.lineplot(tt,x='iteration',y=x,hue='algorithm')
    plt.show()